# <center> Homework 2 </center>

In [1]:
# autoreload позволяет нам подгружать изменения во внешнем коде
# без необходимости перезагружать kernel у ноутбука
%load_ext autoreload
%autoreload 2

## 1) Exceptions
В модуле `exceptions` объявите следующие исключения:
- LowFuelError
- NotEnoughFuel
- CargoOverload

In [2]:
import exceptions

## 2) Vehicle 

<img src="https://cs.pikabu.ru/images/big_size_comm/2013-12_4/13874526438897.jpg" width=300>

Доработайте базовый класс `Vehicle`:
- добавьте атрибуты `weight`, `started`, `fuel`, `fuel_consumption` со значениями по умолчанию
- добавьте инициализатор для установки `weight`, `fuel`, `fuel_consumption`
- добавьте метод `start`, который, если ещё не состояние `started`, проверяет, что топлива больше нуля, 
  и обновляет состояние `started`, иначе выкидывает исключение `exceptions.LowFuelError`
- добавьте метод `move`, который проверяет, 
  что достаточно топлива для преодоления переданной дистанции, 
  и изменяет количество оставшегося топлива, иначе выкидывает исключение `exceptions.NotEnoughFuel`

In [3]:
from abc import ABC, abstractmethod


class Vehicle(ABC):
    
    # добавьте атрибуты weight, started, fuel, fuel_consumption со значениями по умолчанию
    _WEIGHT = 1000
    _STARTED = False
    # _FUEL = 0
    _FUEL_CONSUMPTION = 10
    
    # добавьте инициализатор для установки weight, fuel, fuel_consumption
    @abstractmethod
    def __init__(self, weight = None, fuel = 0, fuel_consupmtion = None):
        # Значения по умолчанию приходится внутри проверять,
        # чтобы можно было свойство класса переопределить при наследовании
        self._weight = float(weight) if not weight is None else self._WEIGHT
        self._started = self._STARTED
        self.fuel = float(fuel)
        self._fuel_consupmtion = float(fuel_consupmtion) if not fuel_consupmtion is None else self._FUEL_CONSUMPTION
    
    # добавьте метод start, который, если ещё не состояние started,
    # проверяет, что топлива больше нуля, и обновляет состояние started,
    # иначе выкидывает исключение exceptions.LowFuelError
    def start(self):
        if not self._started:
            if (self.fuel > 0):
                self._started = True
            else:
                raise exceptions.LowFuelError(f'Current fuel level is {self.fuel}')
            
    def stop(self):
        self._started = False
        
    @property
    def started(self):
        return self._started
    
    # добавьте метод move, который проверяет, что достаточно топлива для преодоления переданной дистанции,
    # и изменяет количество оставшегося топлива,
    # иначе выкидывает исключение exceptions.NotEnoughFuel
    def move(self, distance, stop_arter_moving:bool = False):
        # Проверка переданной дистанции
        distance = float(distance)
        if not (
            # isinstance(distance, (int, float)) and
            distance > 0
        ):
            # Общее исключение для VehicleException, может тут ValueErroe или TypeError вызывать
            raise exceptions.VehicleException(
                'Moving distance must be positive value of int or float. ' + \
                f'Current distance is {distance}, type {type(distance)}'
            )
        
        # Остаток топлива
        rest_fuel = self.fuel - distance * self._fuel_consupmtion / 100 
        
        if rest_fuel < 0: # Если не хваатет
            raise exceptions.NotEnoughFuel(f'Not enough {self.fuel} fuel. Need {rest_fuel * (-1)} fuel more.')
        else:
            self.start() # решил, что не гоже выключенными катиться
            self.fuel = rest_fuel
            if stop_arter_moving: self.stop()
        
        

In [4]:
# Проверки      

class EnyVehicle(Vehicle):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
veh1 = EnyVehicle()
veh1.fuel = 80
veh1.start()
assert veh1.started
veh1.stop()
assert not veh1.started

# veh1.move(-801)
# veh1.move('Far')
# veh1.move()
veh1.move('600')

print(veh1.__dict__)
print(veh1.started)
vars(EnyVehicle)

{'_weight': 1000, '_started': True, 'fuel': 20.0, '_fuel_consupmtion': 10}
True


mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.EnyVehicle.__init__(self, *args, **kwargs)>,
              '__doc__': None,
              '__abstractmethods__': frozenset(),
              '_abc_impl': <_abc._abc_data at 0x2d2772799c0>})

## 3) Engine 

<img src="https://upload.wikimedia.org/wikipedia/commons/6/6f/National_gas_engine_%28Rankin_Kennedy%2C_Modern_Engines%2C_Vol_II%29.jpg" width=250>

Создайте датакласс `Engine`, добавьте атрибуты `volume` и `pistons`

In [5]:
"""
create dataclass `Engine`
"""
from dataclasses import dataclass

@dataclass
class Engine:
    # добавьте атрибуты volume и pistons
    volume: float
    pistons: int

In [6]:
# Проверки      

v6_2_2 = Engine(volume = 2.2, pistons=6)
vars(v6_2_2)

{'volume': 2.2, 'pistons': 6}

## 4) Car 

<img src="https://lh3.googleusercontent.com/proxy/RJqzSJqZFQrx7xOXimZQ4ooInGE6ViJdwz7JZZ_b__Eun-stRH2NMw47fM4je2hL1afRtmpKS9kZUVY0p2YcaZVtJ7bKRw3wWHRRa6zVUbeIb5TYLNkwx5A" width=250>

Cоздайте класс `Car`
    - класс `Car` должен быть наследником `Vehicle`
    - добавьте атрибут `engine` классу `Car`
    - объявите метод `set_engine`, который принимает в себя экземпляр объекта `Engine` и устанавливает на текущий экземпляр `Car`

In [7]:
"""
создайте класс `Car`, наследник `Vehicle`
"""
class Car(Vehicle):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # добавьте атрибут engine классу Car
        self._engine = None
    
    # объявите метод set_engine, который принимает в себя экземпляр объекта Engine
    # и устанавливает на текущий экземпляр Car
    def set_engine(self, newengine:Engine, swap:bool = False):
        if self._engine is None or swap == True:
            self._engine = newengine
        else:
            raise exceptions.VehicleException(
                'Engine could be added only if car have no engine yet ' + \
                'or if it is swap engine operation, called with parameter swap=True. ' + \
                f'Current engine is {vars(self._engine)}'
            )


In [8]:
# Проверки      

car1 = Car()
car1.fuel = 80
car1.start()
assert car1.started
car1.stop()
assert not car1.started

# car1.move(-801)
# car1.move('Far')
# car1.move()
car1.move('600')

car1.set_engine(v6_2_2)
r4_1_4 = Engine(1.4, 4)
car1.set_engine(r4_1_4, swap=True)

print(vars(car1))


class HeavyCar(Car):
    _WEIGHT = 2000
    _FUEL_CONSUMPTION = Vehicle._FUEL_CONSUMPTION * 1.5

car2 = HeavyCar()
car2.set_engine(Engine(5.5, 10))

vars(car2)

{'_weight': 1000, '_started': True, 'fuel': 20.0, '_fuel_consupmtion': 10, '_engine': Engine(volume=1.4, pistons=4)}


{'_weight': 2000,
 '_started': False,
 'fuel': 0.0,
 '_fuel_consupmtion': 15.0,
 '_engine': Engine(volume=5.5, pistons=10)}

## 5) Plane

<img src="http://www.weirduniverse.net/images/2017/1934parachute02.jpg" width=250>

Создайте класс `Plane`
- класс `Plane` должен быть наследником `Vehicle`
- добавьте атрибуты `cargo` и `max_cargo` классу `Plane`
- добавьте `max_cargo` в инициализатор (переопределите родительский)
- объявите метод `load_cargo`, который принимает число, проверяет, что в сумме с текущим `cargo` не будет перегруза, и обновляет значение, в ином случае выкидывает исключение `exceptions.CargoOverload`
- объявите метод `remove_all_cargo`, который обнуляет значение `cargo` и возвращает значение `cargo`, которое было до обнуления

In [9]:
"""
создайте класс `Plane`, наследник `Vehicle`
"""
class Plane(Vehicle):
    _MAX_CARGO = Vehicle._WEIGHT * 0.1
    
    # добавьте атрибуты cargo и max_cargo классу Plane
    # добавьте max_cargo в инициализатор (переопределите родительский)
    def __init__(self, max_cargo = None,  *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cargo = 0
        self._max_cargo = max_cargo if not max_cargo is None else self._MAX_CARGO
    
    @property
    def cargo(self):
        return self._cargo
    
    # объявите метод load_cargo, который принимает число,
    # проверяет, что в сумме с текущим cargo не будет перегруза, и обновляет значение,
    # в ином случае выкидывает исключение exceptions.CargoOverload
    def load_cargo(self, added_cargo):
        added_cargo = float(added_cargo)
        if not added_cargo > 0:
            # Общее исключение для VehicleException, может тут ValueErroe или TypeError вызывать
            raise exceptions.VehicleException(
                'Additional cargo must be positive value of int or float. ' + \
                f'Passed additional cargo is {added_cargo}, type {type(added_cargo)}'
            )
        
        cargo_sum = self._cargo + added_cargo
        
        if cargo_sum <= self._max_cargo:
            self._cargo = cargo_sum
        else:
            raise exceptions.CargoOverload(
                f'Lots of cargo. Current cargo {self._cargo}, additional cargo {added_cargo}. ' + \
                f'Combined cargo {cargo_sum} - it is more then max {self._max_cargo}.'
            )
            
    # объявите метод remove_all_cargo, который обнуляет значение cargo
    # и возвращает значение cargo, которое было до обнуления
    def remove_all_cargo(self):
        removed_cargo = self._cargo
        self._cargo = 0
        return removed_cargo
        
            

In [10]:
# Проверки 

plane1 = Plane()

print(vars(plane1))
print(vars(Plane))

plane1.load_cargo('80')
# plane1.load_cargo(-21)

plane1.fuel = 65
plane1.move(600, stop_arter_moving=True)

plane2 = Plane()
plane2.load_cargo(plane1.remove_all_cargo())

print('\nplane2.cargo:', plane2.cargo)
vars(plane1)

{'_weight': 1000, '_started': False, 'fuel': 0.0, '_fuel_consupmtion': 10, '_cargo': 0, '_max_cargo': 100.0}
{'__module__': '__main__', '_MAX_CARGO': 100.0, '__init__': <function Plane.__init__ at 0x000002D2772A4940>, 'cargo': <property object at 0x000002D2772671A0>, 'load_cargo': <function Plane.load_cargo at 0x000002D2772A4CA0>, 'remove_all_cargo': <function Plane.remove_all_cargo at 0x000002D2772A4D30>, '__doc__': None, '__abstractmethods__': frozenset(), '_abc_impl': <_abc._abc_data object at 0x000002D277283100>}

plane2.cargo: 80.0


{'_weight': 1000,
 '_started': False,
 'fuel': 5.0,
 '_fuel_consupmtion': 10,
 '_cargo': 0,
 '_max_cargo': 100.0}

In [12]:
# Проверка переопределения class property при наследовании
class SuperPlane(Plane):
    _WEIGHT = Plane._WEIGHT * 2
    _MAX_CARGO = _WEIGHT * 0.3
    
    
plane3 = SuperPlane()

vars(plane3)

{'_weight': 2000,
 '_started': False,
 'fuel': 0.0,
 '_fuel_consupmtion': 10,
 '_cargo': 0,
 '_max_cargo': 600.0}